In [1]:
!pip install setuptools==64.0.2
!apt-get install swig
!pip install wrds
!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.5 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  s

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.config_tickers import DOW_30_TICKER
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent,DRLEnsembleAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

In [4]:
from finrl.config_tickers import CSI_300_TICKER

print(CSI_300_TICKER)

['600000.XSHG', '600004.XSHG', '600009.XSHG', '600010.XSHG', '600011.XSHG', '600015.XSHG', '600016.XSHG', '600018.XSHG', '600019.XSHG', '600025.XSHG', '600027.XSHG', '600028.XSHG', '600029.XSHG', '600030.XSHG', '600031.XSHG', '600036.XSHG', '600038.XSHG', '600048.XSHG', '600050.XSHG', '600061.XSHG', '600066.XSHG', '600068.XSHG', '600085.XSHG', '600089.XSHG', '600104.XSHG', '600109.XSHG', '600111.XSHG', '600115.XSHG', '600118.XSHG', '600170.XSHG', '600176.XSHG', '600177.XSHG', '600183.XSHG', '600188.XSHG', '600196.XSHG', '600208.XSHG', '600219.XSHG', '600221.XSHG', '600233.XSHG', '600271.XSHG', '600276.XSHG', '600297.XSHG', '600299.XSHG', '600309.XSHG', '600332.XSHG', '600340.XSHG', '600346.XSHG', '600352.XSHG', '600362.XSHG', '600369.XSHG', '600372.XSHG', '600383.XSHG', '600390.XSHG', '600398.XSHG', '600406.XSHG', '600436.XSHG', '600438.XSHG', '600482.XSHG', '600487.XSHG', '600489.XSHG', '600498.XSHG', '600516.XSHG', '600519.XSHG', '600522.XSHG', '600547.XSHG', '600570.XSHG', '600583.X

In [5]:
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)

check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

# New Section

In [ ]:
print(DOW_30_TICKER)

['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [6]:
TRAIN_START_DATE = '2009-04-01'
TRAIN_END_DATE = '2021-01-01'
TEST_START_DATE = '2021-01-01'
TEST_END_DATE = '2022-06-01'

df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TEST_END_DATE,
                     ticker_list = DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [7]:
df.head()

,date,open,high,low,close,volume,tic,day
0,2009-04-01,3.717500,3.892857,3.710357,3.303860,589372000,AAPL,2
1,2009-04-01,48.779999,48.930000,47.099998,35.911682,10850100,AMGN,2
2,2009-04-01,13.340000,14.640000,13.080000,11.732111,27701800,AXP,2
3,2009-04-01,34.520000,35.599998,34.209999,26.850744,9288800,BA,2
4,2009-04-01,27.500000,29.520000,27.440001,19.726315,15308300,CAT,2


In [8]:
df.sort_values(['date','tic']).head()

,date,open,high,low,close,volume,tic,day
0,2009-04-01,3.717500,3.892857,3.710357,3.303860,589372000,AAPL,2
1,2009-04-01,48.779999,48.930000,47.099998,35.911682,10850100,AMGN,2
2,2009-04-01,13.340000,14.640000,13.080000,11.732111,27701800,AXP,2
3,2009-04-01,34.520000,35.599998,34.209999,26.850744,9288800,BA,2
4,2009-04-01,27.500000,29.520000,27.440001,19.726315,15308300,CAT,2


In [9]:
len(df.tic.unique())

30

In [10]:
df.tic.value_counts()

AAPL    3315
AMGN    3315
WMT     3315
WBA     3315
VZ      3315
V       3315
UNH     3315
TRV     3315
PG      3315
NKE     3315
MSFT    3315
MRK     3315
MMM     3315
MCD     3315
KO      3315
JPM     3315
JNJ     3315
INTC    3315
IBM     3315
HON     3315
HD      3315
GS      3315
DIS     3315
CVX     3315
CSCO    3315
CRM     3315
CAT     3315
BA      3315
AXP     3315
DOW      807
Name: tic, dtype: int64

In [11]:
df.sort_values(['date','tic'],ignore_index=True).head()


,date,open,high,low,close,volume,tic,day
0,2009-04-01,3.717500,3.892857,3.710357,3.303860,589372000,AAPL,2
1,2009-04-01,48.779999,48.930000,47.099998,35.911682,10850100,AMGN,2
2,2009-04-01,13.340000,14.640000,13.080000,11.732111,27701800,AXP,2
3,2009-04-01,34.520000,35.599998,34.209999,26.850744,9288800,BA,2
4,2009-04-01,27.500000,29.520000,27.440001,19.726315,15308300,CAT,2


In [12]:
INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [18]:
from finrl.config import INDICATORS

INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [13]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (3314, 8)
Successfully added vix
Successfully added turbulence index


In [ ]:
processed.head(3)

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-04-01,3.717500,3.892857,3.710357,3.303859,589372000,AAPL,2,0.0,3.537767,3.192146,100.0,66.666667,100.0,3.303859,3.303859,42.279999,0.0
1,2009-04-01,48.779999,48.930000,47.099998,35.911694,10850100,AMGN,2,0.0,3.537767,3.192146,100.0,66.666667,100.0,35.911694,35.911694,42.279999,0.0
2,2009-04-01,13.340000,14.640000,13.080000,11.732111,27701800,AXP,2,0.0,3.537767,3.192146,100.0,66.666667,100.0,11.732111,11.732111,42.279999,0.0


In [28]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)
# processed_full.isna().sum()

In [30]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-04-01,AAPL,3.717500,3.892857,3.710357,3.303860,589372000.0,2.0,0.0,3.537768,3.192147,100.0,66.666667,100.0,3.303860,3.303860,42.279999,0.0
1,2009-04-01,AMGN,48.779999,48.930000,47.099998,35.911682,10850100.0,2.0,0.0,3.537768,3.192147,100.0,66.666667,100.0,35.911682,35.911682,42.279999,0.0
2,2009-04-01,AXP,13.340000,14.640000,13.080000,11.732111,27701800.0,2.0,0.0,3.537768,3.192147,100.0,66.666667,100.0,11.732111,11.732111,42.279999,0.0
3,2009-04-01,BA,34.520000,35.599998,34.209999,26.850744,9288800.0,2.0,0.0,3.537768,3.192147,100.0,66.666667,100.0,26.850744,26.850744,42.279999,0.0
4,2009-04-01,CAT,27.500000,29.520000,27.440001,19.726315,15308300.0,2.0,0.0,3.537768,3.192147,100.0,66.666667,100.0,19.726315,19.726315,42.279999,0.0
5,2009-04-01,CRM,7.815000,8.225000,7.500000,8.172500,13760000.0,2.0,0.0,3.537768,3.192147,100.0,66.666667,100.0,8.172500,8.172500,42.279999,0.0
6,2009-04-01,CSCO,16.510000,17.540001,16.299999,12.300587,73415300.0,2.0,0.0,3.537768,3.192147,100.0,66.666667,100.0,12.300587,12.300587,42.279999,0.0
7,2009-04-01,CVX,66.000000,68.699997,65.830002,39.344143,14299000.0,2.0,0.0,3.537768,3.192147,100.0,66.666667,100.0,39.344143,39.344143,42.279999,0.0
8,2009-04-01,DIS,17.920000,18.820000,17.840000,16.180061,12933600.0,2.0,0.0,3.537768,3.192147,100.0,66.666667,100.0,16.180061,16.180061,42.279999,0.0
9,2009-04-01,GS,104.099998,111.339996,103.940002,89.147743,24883000.0,2.0,0.0,3.537768,3.192147,100.0,66.666667,100.0,89.147743,89.147743,42.279999,0.0


In [31]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

85840
609


In [34]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension

In [35]:
stock_dimension, state_space

(29, 291)